In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [2]:
# importing the models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# Working with Heart Disease Data

In [3]:
heart_data = pd.read_csv("E:/Downloads/Heart data/heart.csv")

In [4]:
heart_data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [5]:
heart_data.shape

(1025, 14)

In [6]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

Splitting the Features and Targets

In [7]:
X = heart_data.drop(columns='target', axis=1)
Y = heart_data['target']

In [8]:
print(X)
print(Y)

      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0      52    1   0       125   212    0        1      168      0      1.0   
1      53    1   0       140   203    1        0      155      1      3.1   
2      70    1   0       145   174    0        1      125      1      2.6   
3      61    1   0       148   203    0        1      161      0      0.0   
4      62    0   0       138   294    1        1      106      0      1.9   
...   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
1020   59    1   1       140   221    0        1      164      1      0.0   
1021   60    1   0       125   258    0        0      141      1      2.8   
1022   47    1   0       110   275    0        0      118      1      1.0   
1023   50    0   0       110   254    0        0      159      0      0.0   
1024   54    1   0       120   188    0        1      113      0      1.4   

      slope  ca  thal  
0         2   2     3  
1         0   0     3  
2  

In [9]:
# Converting into numpy array
Xarr = np.array(X)
Yarr = np.array(Y)

# Model Selection

1. Comparing the values with default hyperparameter values with Cross Validation

In [10]:
# list of models
models = [LogisticRegression(max_iter=1000), SVC(kernel='linear'), RandomForestClassifier(random_state=0), KNeighborsClassifier()]

In [15]:
def evaluate_model(model):
    # perform cross validation
    cv_score = cross_val_score(model, Xarr, Yarr, cv=5)
    print(f"Cross Validation score for {model.__class__.__name__} : {cv_score}")
    return (cv_score.mean()*100).round(2)
# evaluate each model in the list
for model in models:
    score = evaluate_model(model)
    print(f"Accuracy score for {model.__class__.__name__} : {score}%")
    print("-"*150)



Cross Validation score for LogisticRegression : [0.88292683 0.85365854 0.87804878 0.82439024 0.80487805]
Accuracy score for LogisticRegression : 84.88%
------------------------------------------------------------------------------------------------------------------------------------------------------
Cross Validation score for SVC : [0.88292683 0.86829268 0.84390244 0.81463415 0.80487805]
Accuracy score for SVC : 84.29%
------------------------------------------------------------------------------------------------------------------------------------------------------
Cross Validation score for RandomForestClassifier : [1.         1.         0.98536585 1.         0.98536585]
Accuracy score for RandomForestClassifier : 99.41%
------------------------------------------------------------------------------------------------------------------------------------------------------
Cross Validation score for KNeighborsClassifier : [0.76585366 0.74634146 0.76097561 0.71219512 0.75121951]
Accura

1. Comparing the values with different hyperparameter values with GridSearchCV

In [16]:
models_list = [LogisticRegression(max_iter=10000), SVC(), RandomForestClassifier(random_state=0), KNeighborsClassifier()]

In [17]:
# a dictionary to store the hyperparameters for each model
model_hyperparameters = {
    'LogisticRegression': {
        'C': [1, 5, 10],
    },
    'SVC': {
        'C': [1, 5, 10],
        'kernel': ['linear', 'rbf', 'poly']
    },
    'RandomForestClassifier': {
        'n_estimators': [10, 20, 50],
    },
    'KNeighborsClassifier': {
        'n_neighbors': [3, 5, 7],
    }
}

In [18]:
print(model_hyperparameters.keys())

dict_keys(['LogisticRegression', 'SVC', 'RandomForestClassifier', 'KNeighborsClassifier'])


In [20]:
model_keys = list(model_hyperparameters.keys())
print(model_keys)

['LogisticRegression', 'SVC', 'RandomForestClassifier', 'KNeighborsClassifier']


Applying GridSearchCV

In [21]:
def ModelSelection(list_of_model, hyperparameters_dict):
    
    result = []
    i = 0
    for model in list_of_model:
        key = model_keys[i]
        params = hyperparameters_dict[key]
        i += 1
        print(model)
        print(params)
        
        classifier = GridSearchCV(model, params, cv=5)
        
        # fit the model
        classifier.fit(Xarr, Yarr)
        
        result.append({
            'model used ' : model,
            'best score' : classifier.best_score_,
            'best parameters' : classifier.best_params_
        })
        
        result_dataframe = pd.DataFrame(result, columns=['model used ', 'best score', 'best parameters'])
    return result_dataframe

In [22]:
ModelSelection(models_list, model_hyperparameters)

LogisticRegression(max_iter=10000)
{'C': [1, 5, 10]}
SVC()
{'C': [1, 5, 10], 'kernel': ['linear', 'rbf', 'poly']}
RandomForestClassifier(random_state=0)
{'n_estimators': [10, 20, 50]}
KNeighborsClassifier()
{'n_neighbors': [3, 5, 7]}


,model used,best score,best parameters
0,LogisticRegression(max_iter=10000),0.848780,{'C': 1}
1,SVC(),0.844878,"{'C': 5, 'kernel': 'linear'}"
2,RandomForestClassifier(random_state=0),0.997073,{'n_estimators': 20}
3,KNeighborsClassifier(),0.890732,{'n_neighbors': 3}


Random Forest classifier have the highest accuracy with n_estimator = 20